# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This is a classification problem as we need to classify students into two categories:
    * Needs early intervention before they fail to graduate
    * Highly likely to graduate without any intervention

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns) - 1

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed'] == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] == 'no'])

# TODO: Calculate graduation rate
grad_rate = (n_passed * 100.0) / n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300.0

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify = y_all, test_size = (num_test / n_students), random_state = 420)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: ** The three supervised learning models that I feel are appropriate for this problem are:

    * Gaussian Naive Bayes
    * Random Forest Ensemble method
    * AdaBoost Ensemble method

* A model based on Naive Bayes Classifier:
    * Real-world Application:
        * It is used in sentiment analysis (a special application of Natural Language Processing), where the opinion expressed in a sentence is judged to be either postive, negative or neutral towards the main subject of a sentence.
        * References:
            * http://cloudacademy.com/blog/naive-bayes-classifier/
            * http://www.slideshare.net/DevSahu2/sentiment-analysis-using-naive-bayes-classifier-39784368
            * http://www.aclweb.org/anthology/S/S14/S14-2026.pdf
            * https://arxiv.org/pdf/1305.6143.pdf
            * https://web.stanford.edu/~jurafsky/slp3/7.pdf
    * Strengths:
        * It works very fast because it explicitly calculates the needed parameters, thereby not need to perform any iterations.
        * The naive assumpution of the conditional independence of features (when it holds) allows it to model the data with high bias and hence, low variance. So, it works well with less amount of training data. In high variance models, a large amount of training data is required to reduce the error due to variance.
        * It is a generative model (it provides a model of how data is instantiated or generated) by modeling the joint distribution of the various classes. So, it provides a greater insight into the data than discriminative models because Naive Bayes gives information about which class or category is most likely to generate the data in question.
        * It can easily perform online updates to the model. So, if any new training data becomes available over the course of the models' use in production, the model can be easily "fitted" to accomodate the new training data.
    * Weaknesses:
        * The naive assumption of the conditional independence of features does not generally hold in practice.
        * If the training data does not contain any occurrence of a combination of a certain attribute value and a certain class label, then the frequency-based probability estimate for that combination will be zero. In other words, this issue occurs when the training sample taken from a population is not representative of the entire population.
    * Reasons to use this model:
        * The amount of data is less so this model should work well.
        * It is highly possible that one of the features contributed to the student not being able to graduate. So, the naive assumpution of the conditional independence of features can be assumed here.
        * This model is fast to build so it should work well in this case where we have a large number of features.

* A model based on Random Forest Ensemble Classifier
    * Real-world Application:
        * Random Forests have been used in Genome-wide association study, where the genomes (complete DNA sequences) of different samples have been compared to provide critical information about different diseases. This gives the suceptibility of each different individual to complex diseases as well as the response to different drugs based on the individual's genetic variations.
        * References:
            * https://www.cs.cmu.edu/~qyj/papersA08/11-rfbook.pdf
            * http://rp-www.cs.usyd.edu.au/~yangpy/publication/EnsembleBioinformatics-v6.pdf
    * Strengths:
        * It has all the strengths of Decision Trees without many of their weaknesses. Due to the efficient use of the bagging technique used in Random Forests, the model has less variance as compared of individual Decision Trees which helps prevent overfitting to the training data.
        * It has only one important parameter which needs to be tuned (the number of features to randomly select at each node), thereby making it an easy to setup model.
        * It is fast during the training step because it can run each decision tree training in parallel.
        * It does not require the data to be linearly separable.
        * It can handle small or large number of features as well as small or large number of training examples very well.
        * It provides a really good idea of which features in the dataset are the most important.
        * It requires very little data preparation like normalization, creation of dummy variable etc. because it can work with both numerical values and categorical values.
    * Weaknesses:
        * If the number of trees is large, then the prediction step may be slow.
        * It is very difficult to perform online learning. If any new training data is available, it is better to simply perform the training step again on the entire data, thereby giving a new model.
        * It is difficult to understand the result of the learning of a Random Forest model as compared to a Decision Tree model (which follows simple boolean logic).
    * Reasons to use this model:
        * There are a large number of features in the provided data so a Random Forest will perform well.
        * Random Forests are good at choosing which features are important to arrive at the prediction, thereby ignoring features which do not contribute enough to the final prediction. Out of the many features available, not all would be an important factor in the graduation rate of the students.
        * A Random Forest is a generally robust model that is not much prone to overfitting, which would be beneficial in this case as the number of available training examples is low.

* A model based on AdaBoost Ensemble Classifier:
    * Real-World Application:
        * AdaBoost is used in Face Detection software, where the software identifies the name of the person in the given image.
        * References:
            * http://pages.ucsd.edu/~ztu/publication/dollarBMVC09ChnFtrs_0.pdf
            * http://lbmedia.ece.ucsb.edu/research/human/human.htm
            * http://www.uni-obuda.hu/journal/Markoski_Ivankovic_Ratgeber_Pecev_Glusac_57.pdf
            * https://www.cs.utexas.edu/~grauman/courses/spring2007/395T/papers/viola_cvpr2001.pdf
    * Strengths:
        * It has only one important parameter which needs to be tuned (the number of iterations to perform), thereby making it an easy to setup model.
        * It is very flexible and can combine with a large variety of training algorithms.
        * It is effective because it only needs to find classifiers which are better than random guessing.
        * It can work with numeric as well as non-numeric data.
    * Weaknesses:
        * It is sensitive to noisy data and outliers.
        * It can sometimes provide sub-optimal solutions.
    * Reasons to use this model:
        * It can help to select the subset of the important features in an iterative manner, thereby building on the success / failure of the previous iteration. This is important in this case of the large number of features available.
        * It is fast and provides high quality models, something that will help in this case where the number of features is large and the amount of training data is not large.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = RandomForestClassifier(random_state = 420)
clf_C = AdaBoostClassifier(random_state = 420)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
    print "\n{}: \n".format(clf.__class__.__name__)
    for n in [100, 200, 300]:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)


GaussianNB: 

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0009 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.5000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.3294.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0006 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.7669.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7576.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0006 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.7854.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7385.

RandomForestClassifier: 

Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0128 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 0.9931.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.7222.
Training a RandomForestCla

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0009           |         0.0002         |      0.5000      |     0.3294      |
| 200               |        0.0006           |         0.0002         |      0.7669      |     0.7576      |
| 300               |        0.0006           |         0.0002         |      0.7854      |     0.7385      |

** Classifer 2 - RandomForestClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0128           |         0.0007         |      0.9931      |     0.7222      |
| 200               |        0.0437           |         0.0006         |      0.9920      |     0.7344      |
| 300               |        0.0138           |         0.0006         |      0.9926      |     0.7612      |

** Classifer 3 - AdaBoostClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0562           |         0.0031         |      0.9931      |     0.7391      |
| 200               |        0.0620           |         0.0033         |      0.8485      |     0.7445      |
| 300               |        0.0675           |         0.0031         |      0.8638      |     0.7943      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** I choose AdaBoostClassifier as the best model for this dataset. There are several reasons for choosing this model:

* The most important reason for choosing the AdaBoostClassifier model is it's F1 score on the test dataset (unseen data), which is higher than that of the other two models.
* The training and prediction times of the AdaBoostClassifier model are slightly higher than that of the other two models. However, this disadvantage is offset by the accuracy of the predictive power of the AdaBoostClassifier model on the current dataset. Also, the training time is less than a fraction of a second so this model is cost-effective to train with respect to the limited resources available.
* Based on the comparision of the F1 scores of the test and training dataset, the AdaBoostClassifier model is less prone to over-fitting as compared to the RandomForestClassifier model.
* Based on the comparision of the F1 scores of the test and training dataset, the AdaBoostClassifier model is less prone to under-fitting as compared to the GaussianNB model.
* The AdaBoostClassifier model is working better than the other two models with the limited amount of training data that is currently available.
* The number of parameters that need to be tuned for the AdaBoostClassifer is less as compared to the RadomForestClassifier model. So, the amount of time spent trying out various values for the parameters to tune the model will be less as compared to the other two models. The GaussianNB model does not even provide any parameters to tune the model.
* The results of the AdaBoostClassfier model may not be as easily interpretable as that of the GaussianNB (which is just a collection of class probabilities). However, this disadvantage is, again, offset by the accuracy of the predictive power of the AdaBoostClassifier model on the current dataset.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

* The AdaBoostClassifier model begins by learning a list of instructions which explain the relationship of the factors in a student's life (like parents' education status, the family's financial stability etc.) to the final result whether the student has passed or failed. The list of instructions together can provide a prediction of pass or fail, based on the values of the various factors.
* Each prediction on a student's result has some weight associated with it. This weight is used as an indicator of the importance of that particular prediction. Predictions with higher weights are given more priority than predictions with lower weight. Initially, all the predictions have equal weight. In other words, this weight is indicative of the closeness of the student's result to the results of the general population of students.
* The model uses the above learned rules on the same data that it used during the above learning phase to see if it can predict the final result of pass or fail accurately for each student.
* In cases where the above prediction is incorrect, it increases the weights of the incorrect predictions and then tries to make predictions again on the same data. This way, any incorrect predictions get more weightage in the next run and thus, these incorrect predictions become correct predictions on the next run of predictions on the same data.
* The above process is repeated a certain number of times. This causes subsequent runs of the predictions to focus more on the difficult cases where it got incorrect predictions.
* It is possible that the next run of predictions might cause an earlier correctly predicted result to be incorrectly predicted. Now, since this prediction is incorrect in the current run, it will get more weightage in the next run, thereby correctly predicting it in the subsequent runs.
* So, the AdaBoostClassifier model iteratively corrects it's own mistakes.
* In some ways, this is exactly how a teacher might teach a class of say, 50 students. After every test, the teacher will focus more attention in class on the students who got low marks in the previous test, to ensure that they pass the next one.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [11]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'n_estimators': [i for i in range(6, 11)],
              'learning_rate': [0.1, 0.25, 0.5, 0.75, 1.0],
              'base_estimator__max_depth' : [i for i in range(1, 6)]
             }

# TODO: Initialize the classifier
from sklearn.tree import DecisionTreeClassifier

clf = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(), random_state = 420)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring = f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
print ""
print "Best value for the parameters used by the tuned model: {}.".format(grid_obj.best_params_)

Made predictions in 0.0009 seconds.
Tuned model has a training F1 score of 0.8515.
Made predictions in 0.0008 seconds.
Tuned model has a testing F1 score of 0.8188.

Best value for the parameters used by the tuned model: {'n_estimators': 9, 'learning_rate': 0.5, 'base_estimator__max_depth': 1}.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The F1 scores of the final tuned model are:

    * Training: 0.8515
    * Testing: 0.8188

* The training F1 score has reduced from 0.8638 (untuned model, 300 training samples) to 0.8515 (tuned model, 300 training samples). Thus, the accuracy of the predictions on the training dataset has decreased as more training samples are provided.
* The testing F1 score has increased from 0.7943 (untuned model, 300 training samples) to 0.8188 (tuned model, 300 training samples). Thus, the accuracy of the predictions on the test dataset has increased as more training samples are provided.

The untuned model seems to have slightly high variance and slightly high bias. As a result of this, the difference of F1 scores of the training dataset and the testing dataset in the untuned model is larger as compared to the difference of the F1 scores of the tuned model.

The tuning of the model parameters has resulted in decreasing the variance and the bias of the model, leading to better predictions on the test dataset. This is evident in the two F1 scores:

* The poorer F1 score on the training dataset of the tuned model as compared to the F1 score on the training dataset of the untuned model indicates that the variance of the model has reduced.
* The better F1 score on the test dataset of the tuned model as compared to the F1 score on the test dataset of the untuned model indicates that the bias of the model has reduced.

Both the above F1 score changes show that the variance and the bias of the model has decreased in a positive manner for the test dataset.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.